In [1]:
from IPython.core.display import display, HTML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import helper_functions as hf

warnings.filterwarnings('ignore')

In [2]:
pos = pd.read_csv('POS_CASH_balance.csv').sort_values(['SK_ID_PREV','MONTHS_BALANCE'])
pos['NAME_CONTRACT_STATUS'] = pos['NAME_CONTRACT_STATUS'].replace(['XNA','Canceled','Amortized debt'],'OTHER')
pos = pd.get_dummies(pos)
pos.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_OTHER,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed
6030662,1000001,158271,-10,12.0,12.0,0,0,1,0,0,0,0,0,0
8470736,1000001,158271,-9,12.0,11.0,0,0,1,0,0,0,0,0,0
45995,1000001,158271,-8,2.0,0.0,0,0,0,0,1,0,0,0,0
4467804,1000002,101962,-54,4.0,4.0,0,0,1,0,0,0,0,0,0
3346102,1000002,101962,-53,6.0,3.0,0,0,1,0,0,0,0,0,0


The one categorical feature in this dataset is NAME_CONTRACT_STATUS. The categories 'XNA', 'Canceled', and 'Amortized debt' are sparse and are all grouped into another category 'OTHER' before one hot encoding this feature. 

In [3]:
pos_prev_aggs_dict = {}
for i in pos.columns[3:]:
    if i == 'CNT_INSTALMENT':
        pos_prev_aggs_dict[i] = ['nunique','count']
    if i == 'CNT_INSTALMENT_FUTURE':
        pos_prev_aggs_dict[i] = ['min','max','mean','sum']
    if i.startswith('SK_DPD'):
        pos_prev_aggs_dict[i] = ['max','mean','sum','var']
    else:
        pos_prev_aggs_dict[i] = ['max','mean','var']
pos_prev_aggs = hf.get_aggs_df(pos,'SK_ID_PREV',pos_prev_aggs_dict)
pos_prev_aggs.head()

100.0 %


,CNT_INSTALMENT_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_var,CNT_INSTALMENT_FUTURE_max,CNT_INSTALMENT_FUTURE_mean,CNT_INSTALMENT_FUTURE_var,SK_DPD_max,SK_DPD_mean,SK_DPD_sum,SK_DPD_var,...,NAME_CONTRACT_STATUS_Demand_var,NAME_CONTRACT_STATUS_OTHER_max,NAME_CONTRACT_STATUS_OTHER_mean,NAME_CONTRACT_STATUS_OTHER_var,NAME_CONTRACT_STATUS_Returned to the store_max,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Returned to the store_var,NAME_CONTRACT_STATUS_Signed_max,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_Signed_var
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,
1000001,12.0,8.666667,33.333333,12.0,7.666667,44.333333,0,0.0,0,0.0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
1000002,6.0,5.200000,1.200000,4.0,2.000000,2.500000,0,0.0,0,0.0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
1000003,12.0,12.000000,0.000000,12.0,10.500000,1.666667,0,0.0,0,0.0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
1000004,10.0,9.625000,1.125000,10.0,6.125000,10.125000,0,0.0,0,0.0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
1000005,10.0,10.000000,0.000000,10.0,5.000000,11.000000,0,0.0,0,0.0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0


In [4]:
pos_prev_slopes = hf.get_slopes(pos,pos['SK_ID_PREV'].unique(),'SK_ID_PREV','MONTHS_BALANCE',pos.columns[4:])
pos_prev_slopes.to_csv('pos_prev_slopes.csv',columns = pos_prev_slopes.columns)
pos_prev_slopes = pd.read_csv('pos_prev_slopes.csv').set_index('SK_ID_PREV')
pos_prev_slopes.head()

99.965 %
0
Found slopes in Index(['CNT_INSTALMENT_FUTURE', 'SK_DPD', 'SK_DPD_DEF',
       'NAME_CONTRACT_STATUS_Active', 'NAME_CONTRACT_STATUS_Approved',
       'NAME_CONTRACT_STATUS_Completed', 'NAME_CONTRACT_STATUS_Demand',
       'NAME_CONTRACT_STATUS_OTHER',
       'NAME_CONTRACT_STATUS_Returned to the store',
       'NAME_CONTRACT_STATUS_Signed'],
      dtype='object') grouped by SK_ID_PREV with respect to MONTHS_BALANCE


,CNT_INSTALMENT_FUTURE_slope,SK_DPD_slope,SK_DPD_DEF_slope,NAME_CONTRACT_STATUS_Active_slope,NAME_CONTRACT_STATUS_Approved_slope,NAME_CONTRACT_STATUS_Completed_slope,NAME_CONTRACT_STATUS_Demand_slope,NAME_CONTRACT_STATUS_OTHER_slope,NAME_CONTRACT_STATUS_Returned to the store_slope,NAME_CONTRACT_STATUS_Signed_slope
SK_ID_PREV,,,,,,,,,,
1000001,-6.00,0.0,0.0,-0.500000,0.0,0.500000,0.0,0.0,0.0,0.0
1000002,-1.00,0.0,0.0,-0.200000,0.0,0.200000,0.0,0.0,0.0,0.0
1000003,-1.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1000004,-1.25,0.0,0.0,-0.083333,0.0,0.083333,0.0,0.0,0.0,0.0
1000005,-1.00,0.0,0.0,-0.045455,0.0,0.045455,0.0,0.0,0.0,0.0


In [5]:
pos_prev_weighted = hf.get_weighted(pos,'SK_ID_PREV','MONTHS_BALANCE',pos.columns[4:])
pos_prev_weighted.to_csv('pos_prev_weighted.csv',columns = pos_prev_weighted.columns)
pos_prev_weighted = pd.read_csv('pos_prev_weighted.csv').set_index('SK_ID_PREV')
pos_prev_weighted.head()

,CNT_INSTALMENT_FUTURE_weighted,SK_DPD_weighted,SK_DPD_DEF_weighted,NAME_CONTRACT_STATUS_Active_weighted,NAME_CONTRACT_STATUS_Approved_weighted,NAME_CONTRACT_STATUS_Completed_weighted,NAME_CONTRACT_STATUS_Demand_weighted,NAME_CONTRACT_STATUS_OTHER_weighted,NAME_CONTRACT_STATUS_Returned to the store_weighted,NAME_CONTRACT_STATUS_Signed_weighted
SK_ID_PREV,,,,,,,,,,
1000001,6.419810,0.0,0.0,0.563791,0.0,0.436209,0.0,0.0,0.0,0.0
1000002,1.588395,0.0,0.0,0.727665,0.0,0.272335,0.0,0.0,0.0,0.0
1000003,10.182866,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1000004,5.309654,0.0,0.0,0.828366,0.0,0.171634,0.0,0.0,0.0,0.0
1000005,4.081010,0.0,0.0,0.875675,0.0,0.124325,0.0,0.0,0.0,0.0


In [12]:
pos_prev = pd.concat([pos_prev_weighted,pos_prev_aggs,pos_prev_slopes,],axis=1)
pos_prev['SK_ID_CURR'] = pos.groupby('SK_ID_PREV').max()['SK_ID_CURR']
pos_prev.to_csv('pos_prev.csv',columns = pos_prev.columns)
pos_prev = pd.read_csv('pos_prev.csv').set_index('SK_ID_CURR')
pos_prev.head()

,CNT_INSTALMENT_FUTURE_weighted,SK_DPD_weighted,SK_DPD_DEF_weighted,NAME_CONTRACT_STATUS_Active_weighted,NAME_CONTRACT_STATUS_Approved_weighted,NAME_CONTRACT_STATUS_Completed_weighted,NAME_CONTRACT_STATUS_Demand_weighted,NAME_CONTRACT_STATUS_OTHER_weighted,NAME_CONTRACT_STATUS_Returned to the store_weighted,NAME_CONTRACT_STATUS_Signed_weighted,...,SK_DPD_slope,SK_DPD_DEF_slope,NAME_CONTRACT_STATUS_Active_slope,NAME_CONTRACT_STATUS_Approved_slope,NAME_CONTRACT_STATUS_Completed_slope,NAME_CONTRACT_STATUS_Demand_slope,NAME_CONTRACT_STATUS_OTHER_slope,NAME_CONTRACT_STATUS_Returned to the store_slope,NAME_CONTRACT_STATUS_Signed_slope,SK_ID_CURR
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,
1000001,6.419810,0.0,0.0,0.563791,0.0,0.436209,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.500000,0.0,0.500000,0.0,0.0,0.0,0.0,158271
1000002,1.588395,0.0,0.0,0.727665,0.0,0.272335,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.200000,0.0,0.200000,0.0,0.0,0.0,0.0,101962
1000003,10.182866,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,252457
1000004,5.309654,0.0,0.0,0.828366,0.0,0.171634,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.083333,0.0,0.083333,0.0,0.0,0.0,0.0,260094
1000005,4.081010,0.0,0.0,0.875675,0.0,0.124325,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.045455,0.0,0.045455,0.0,0.0,0.0,0.0,176456


In [16]:
pos_curr_aggs_dict = {}
for i in pos_prev.columns[:-1]:
    if i == 'CNT_INSTALMENT':
        pos_curr_aggs_dict[i] = ['std','var']
    if i == 'CNT_INSTALMENT_FUTURE':
        pos_curr_aggs_dict[i] = ['min','max','mean','sum']
    if i.startswith('SK_DPD'):
        pos_curr_aggs_dict[i] = ['max','mean','sum']
    else:
        pos_curr_aggs_dict[i] = ['min','max','mean','sum']
pos_curr_aggs = hf.get_aggs_df(pos_prev,'SK_ID_CURR',pos_curr_aggs_dict)
pos_curr_aggs.head()

100.0 %


,CNT_INSTALMENT_FUTURE_weighted_min,CNT_INSTALMENT_FUTURE_weighted_max,CNT_INSTALMENT_FUTURE_weighted_mean,CNT_INSTALMENT_FUTURE_weighted_sum,SK_DPD_weighted_max,SK_DPD_weighted_mean,SK_DPD_weighted_sum,SK_DPD_DEF_weighted_max,SK_DPD_DEF_weighted_mean,SK_DPD_DEF_weighted_sum,...,NAME_CONTRACT_STATUS_OTHER_slope_mean,NAME_CONTRACT_STATUS_OTHER_slope_sum,NAME_CONTRACT_STATUS_Returned to the store_slope_min,NAME_CONTRACT_STATUS_Returned to the store_slope_max,NAME_CONTRACT_STATUS_Returned to the store_slope_mean,NAME_CONTRACT_STATUS_Returned to the store_slope_sum,NAME_CONTRACT_STATUS_Signed_slope_min,NAME_CONTRACT_STATUS_Signed_slope_max,NAME_CONTRACT_STATUS_Signed_slope_mean,NAME_CONTRACT_STATUS_Signed_slope_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0.519041,1.588395,1.053718,2.107437,1.606329,0.803164,1.606329,1.606329,0.803164,1.606329,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
100002,13.490387,13.490387,13.490387,13.490387,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
100003,1.996189,6.966387,4.821900,14.465701,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
100004,1.846690,1.846690,1.846690,1.846690,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
100005,6.257201,6.257201,6.257201,6.257201,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.045455,-0.045455,-0.045455,-0.045455


In [ ]:
# pos_curr_slopes = hf.get_slopes(pos_prev,pos_prev['SK_ID_CURR'].unique(),'SK_ID_CURR','MONTHS_BALANCE',pos_prev.columns[:-1])
# pos_curr_slopes.to_csv('pos_curr_slopes',columns = pos_curr_slopes.columns)
# pos_curr_slopes = pd.read_csv('pos_curr_slopes.csv').set_index('SK_ID_CURR')
# pos_curr_slopes.head()

In [ ]:
pos_curr_weighted = hf.get_weighted(pos,'SK_ID_CURR','MONTHS_BALANCE',pos_prev.columns[:-1])
pos_curr_weighted.to_csv('pos_curr_weighted.csv',columns = pos_curr_weighted.columns)
pos_curr_weighted = pd.read_csv('pos_curr_weighted.csv').set_index('SK_ID_CURR')
pos_curr_weighted.head()